In [ ]:
# Import all libraries

In [372]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix

In [ ]:
# Read in dataframe

In [290]:
df = pd.read_csv('fbdh1.csv')

In [346]:
# df.head()

In [ ]:
# Use a label fix function to assign classifications or ground truth to Flow thresholds

In [348]:
def label_fix(label):
    if label < -10:
        return 0
    elif label > 10:
        return 1
    else:
        return 2

df['Class'] = df['Flow'].apply(label_fix)

In [349]:
df.head()

,Flow,D1,D2,P1,P2,dD1,dD2,dP1,dP2,Class
0,-16.69,99.49,-145.71,48.04,-15.90,81.17,12.26,36.71,2.21,0
1,-16.23,172.78,-118.66,79.84,-11.94,80.09,18.49,35.48,2.91,0
2,-15.54,242.66,-81.28,109.77,-7.75,75.74,24.65,33.03,3.26,0
3,-15.17,316.42,-55.50,140.84,-5.30,74.07,28.43,31.89,3.37,0
4,-14.51,392.40,-31.67,171.92,-1.24,73.96,29.70,31.32,3.68,0


In [ ]:
# Create the Feature matrix

In [350]:
X = df.drop('Flow', axis = 1)
X.drop('Class', axis = 1, inplace = True)

In [351]:
X.head()

,D1,D2,P1,P2,dD1,dD2,dP1,dP2
0,99.49,-145.71,48.04,-15.90,81.17,12.26,36.71,2.21
1,172.78,-118.66,79.84,-11.94,80.09,18.49,35.48,2.91
2,242.66,-81.28,109.77,-7.75,75.74,24.65,33.03,3.26
3,316.42,-55.50,140.84,-5.30,74.07,28.43,31.89,3.37
4,392.40,-31.67,171.92,-1.24,73.96,29.70,31.32,3.68


In [ ]:
# Scale the features

In [352]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [353]:
scaler.fit(X)
scaled_features = scaler.transform(X)
X = pd.DataFrame(scaled_features, columns = X.columns[:])

In [ ]:
# Create the classification matrix

In [354]:
y = df['Class']

In [ ]:
# Perform train test split

In [355]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [ ]:
# Name columns in tensorflow

In [356]:
D1 = tf.feature_column.numeric_column('D1')
D2 = tf.feature_column.numeric_column('D2')
P1 = tf.feature_column.numeric_column('P1')
P2 = tf.feature_column.numeric_column('P2')
dD1 = tf.feature_column.numeric_column('dD1')
dD2 = tf.feature_column.numeric_column('dD2')
dP1 = tf.feature_column.numeric_column('dP1')
dP2 = tf.feature_column.numeric_column('dP2')

In [357]:
feat_cols = [D1, D2, P1, P2, dD1, dD2, dP1, dP2]

In [ ]:
# Create input function for Deep Learning

In [358]:
input_func = tf.estimator.inputs.pandas_input_fn(x = X_train, y = y_train, 
                                                 batch_size = 500, num_epochs = None, shuffle = True)

In [ ]:
# Create Linear Estimator

In [359]:
Linear = tf.estimator.LinearClassifier(n_classes = 3, feature_columns = feat_cols)

W0903 22:08:01.217786 4707644864 estimator.py:1811] Using temporary folder as model directory: /var/folders/j0/8hdcgtwn33v9lp5kvxshrvkm0000gn/T/tmp7qbl07ew


In [ ]:
# Create DNN Estimator

In [360]:
classifier = tf.estimator.DNNClassifier(hidden_units = [10, 20, 20, 10], n_classes = 3, feature_columns = feat_cols)

W0903 22:08:02.522065 4707644864 estimator.py:1811] Using temporary folder as model directory: /var/folders/j0/8hdcgtwn33v9lp5kvxshrvkm0000gn/T/tmppzsezzsu


In [ ]:
# Train Linear Estimator

In [361]:
Linear.train(input_fn = input_func, steps = 5000)

In [ ]:
# Train DNN Estimator

In [362]:
classifier.train(input_fn = input_func, steps = 5000)

In [ ]:
# Creation Predictions Function

In [363]:
pred_fn = tf.estimator.inputs.pandas_input_fn(x = X_test, batch_size = len(X_test), shuffle = False)

In [ ]:
# Generate note of Linear and DNN Predictions

In [364]:
note_pred_linear = list(Linear.predict(input_fn = pred_fn))

In [365]:
note_predictions = list(classifier.predict(input_fn=pred_fn))

In [ ]:
# Gather Predictions in Matrices

In [366]:
final_linear_preds = []
for pred in note_pred_linear:
    final_linear_preds.append(pred['class_ids'][0])

In [367]:
final_preds  = []
for pred in note_predictions:
    final_preds.append(pred['class_ids'][0])

In [ ]:
# Classification Report Linear Classifier

In [369]:
print(classification_report(y_test,final_linear_preds))

              precision    recall  f1-score   support

           0       0.92      0.90      0.91       882
           1       0.93      0.93      0.93       814
           2       0.63      0.67      0.65       304

   micro avg       0.88      0.88      0.88      2000
   macro avg       0.83      0.83      0.83      2000
weighted avg       0.88      0.88      0.88      2000



In [ ]:
# Classification Report DNN Classifier

In [370]:
print(classification_report(y_test, final_preds))

              precision    recall  f1-score   support

           0       0.96      0.94      0.95       882
           1       0.92      0.94      0.93       814
           2       0.72      0.73      0.72       304

   micro avg       0.90      0.90      0.90      2000
   macro avg       0.87      0.87      0.87      2000
weighted avg       0.91      0.90      0.90      2000



In [373]:
# Convert Tensorflow Model to Tensorflow Lite

In [ ]:
# converter = tf.lite.TFLiteConverter.from_saved_model('/Users/mikefurr/Downloads')